In [3]:
import sys
import typing as tp
import os

import pandas as pd
import plotly.express as px

In [4]:
sys.path.append(
    os.path.join(
        os.path.dirname(os.path.dirname(__vsc_ipynb_file__)),
        "build",
        "pacer",
        "gps-source",
    )
)

from _pacer_gps_source_impl import (
    GPSSample,
    GPSSource,
    GPMFSource,
    SequentialGPSSource,
)

In [5]:
def read_all_gps_data(s: GPSSource) -> tp.Iterable[tuple[GPSSample, float, float]]:
    """
    Read all GPS data from the given GPSSource and return it as a pandas DataFrame.
    """
    gps_data = []
    while not s.IsEnd():
        s.Samples(lambda s, i, n: gps_data.append(s))
        for si in gps_data:
            yield si, *s.CurrentTimeSpan()
        gps_data.clear()
        s.Next()

In [6]:
s = GPMFSource("/Users/denys/Documents/gokarting-ui/GH010219.MP4")

In [7]:
df = pd.DataFrame(
    {
        "latitude": si.latitude,
        "longitude": si.longitude,
        "altitude": si.altitude,
        "3d_speed": si.full_speed * 3.6,
        "2d_speed": si.ground_speed * 3.6,
        "time_in": t0,
        "time_out": t1,
    }
    for si, t0, t1 in read_all_gps_data(s)
)

df

,latitude,longitude,altitude,3d_speed,2d_speed,time_in,time_out
0,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
1,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
2,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
3,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
4,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
...,...,...,...,...,...,...,...
9651,51.375913,-0.361280,20.541,40.1976,39.888,531.0,532.0
9652,51.375908,-0.361276,20.585,40.1904,40.212,531.0,532.0
9653,51.375904,-0.361271,20.609,39.9636,40.212,531.0,532.0
9654,51.375899,-0.361266,20.637,39.5460,39.996,531.0,532.0


In [8]:
fig = px.scatter(
    df.loc[lambda d: d["3d_speed"] > 0.1],
    x="longitude",
    y="latitude",
    color="3d_speed",
    color_continuous_scale="Viridis",
    title="GPS Data (Colored by 3D Speed)",
)
fig

In [9]:
px.line(df["3d_speed"], title="3D Speed over Time")